In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import itertools

In [ ]:
N_ITERATIONS = 150
EPOCHS_PER_ITERATION = 8000
N_ADD_POINTS = 2000

N_PDE_POINTS_PARAMETRIC = 16384
N_BC_POINTS_PARAMETRIC = 8192

BEST_PARAMS = {
    'learning_rate': 0.000019,
    'num_layers': 8,
    'hidden_size': 384,
    'W_DIRICHLET': 36.163964,
    'W_NEUMANN': 1.926002,
    'W_OVERSHOOT': 154.014293,
    'W_DATA': 162.695670,
    'W_E_FIELD_X': 6.170429,
    'W_E_FIELD_Y': 17.506671,
}

LEARNING_RATE = BEST_PARAMS['learning_rate']
W_DIRICHLET = BEST_PARAMS['W_DIRICHLET']
W_NEUMANN = BEST_PARAMS['W_NEUMANN']
W_OVERSHOOT = BEST_PARAMS['W_OVERSHOOT']
W_DATA = BEST_PARAMS['W_DATA']
W_E_FIELD_X = BEST_PARAMS['W_E_FIELD_X']
W_E_FIELD_Y = BEST_PARAMS['W_E_FIELD_Y']
NUM_LAYERS = BEST_PARAMS['num_layers']
HIDDEN_SIZE = BEST_PARAMS['hidden_size']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

In [ ]:
L_range = [40.0, 110.0]; w_range = [2.0, 6.0]; d_range = [2.0, 8.0]
lx, ly = 120.0, 22.5
x_domain_phys = [-lx / 2, lx / 2]; y_domain_phys = [-ly / 2, ly / 2]
V_top = 1.0; V_bottom = 0.0

L_char_x = lx / 2.0
L_char_y = ly / 2.0

lx_n, ly_n = 2.0, 2.0
x_domain_n = [-1.0, 1.0]; y_domain_n = [-1.0, 1.0]

In [ ]:
class ParametricPINN(nn.Module):
    def __init__(self, num_layers=NUM_LAYERS, hidden_size=HIDDEN_SIZE):
        super(ParametricPINN, self).__init__()
        layers = [nn.Linear(5, hidden_size), nn.Tanh()]
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_size, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x_n, y_n, L_n, w_n, d_n):
        batch_size = x_n.shape[0]
        if L_n.numel() == 1: L_n = L_n.expand(batch_size, 1)
        if w_n.numel() == 1: w_n = w_n.expand(batch_size, 1)
        if d_n.numel() == 1: d_n = d_n.expand(batch_size, 1)
        input_features = torch.cat([
            x_n.view(-1, 1), y_n.view(-1, 1),
            L_n.view(-1, 1), w_n.view(-1, 1), d_n.view(-1, 1)
        ], dim=1)
        return self.net(input_features)

In [ ]:
def is_inside_plates_parametric(x_n, y_n, L_n, w_n, d_n, L_char_x, L_char_y):
    scale_ratio = L_char_x / L_char_y
    w_n_y_scaled = w_n * scale_ratio
    d_n_y_scaled = d_n * scale_ratio
    
    plate_x_min_n = -L_n / 2; plate_x_max_n = L_n / 2
    top_plate_y_min_n = d_n_y_scaled / 2; top_plate_y_max_n = d_n_y_scaled / 2 + w_n_y_scaled
    bottom_plate_y_min_n = -d_n_y_scaled / 2 - w_n_y_scaled; bottom_plate_y_max_n = -d_n_y_scaled / 2
    
    in_top = (x_n >= plate_x_min_n) & (x_n <= plate_x_max_n) & (y_n >= top_plate_y_min_n) & (y_n <= top_plate_y_max_n)
    in_bottom = (x_n >= plate_x_min_n) & (x_n <= plate_x_max_n) & (y_n >= bottom_plate_y_min_n) & (y_n <= bottom_plate_y_max_n)
    return in_bottom | in_top

def sample_params(batch_size, device, L_char_x):
    L_phys = torch.rand(batch_size, 1, device=device) * (L_range[1] - L_range[0]) + L_range[0]
    w_phys = torch.rand(batch_size, 1, device=device) * (w_range[1] - w_range[0]) + w_range[0]
    d_phys = torch.rand(batch_size, 1, device=device) * (d_range[1] - d_range[0]) + d_range[0]
    return L_phys / L_char_x, w_phys / L_char_x, d_phys / L_char_x

def generate_pde_points_parametric(n_points, device, L_char_x, L_char_y):
    x_pde_n_list, y_pde_n_list, L_n_list, w_n_list, d_n_list = [], [], [], [], []
    num_generated = 0
    while num_generated < n_points:
        num_candidates = int(n_points * 1.5) + 1
        L_cand_n, w_cand_n, d_cand_n = sample_params(num_candidates, device, L_char_x)
        x_cand_n = torch.rand(num_candidates, 1, device=device) * 2 - 1
        y_cand_n = torch.rand(num_candidates, 1, device=device) * 2 - 1
        is_outside = ~is_inside_plates_parametric(x_cand_n, y_cand_n, L_cand_n, w_cand_n, d_cand_n, L_char_x, L_char_y)
        x_pde_n_list.append(x_cand_n[is_outside]); y_pde_n_list.append(y_cand_n[is_outside])
        L_n_list.append(L_cand_n[is_outside]); w_n_list.append(w_cand_n[is_outside]); d_n_list.append(d_cand_n[is_outside])
        num_generated += is_outside.sum().item()
    x_pde = torch.cat(x_pde_n_list, dim=0)[:n_points]; y_pde = torch.cat(y_pde_n_list, dim=0)[:n_points]
    L_pde = torch.cat(L_n_list, dim=0)[:n_points]; w_pde = torch.cat(w_n_list, dim=0)[:n_points]; d_pde = torch.cat(d_n_list, dim=0)[:n_points]
    return (x_pde, y_pde), (L_pde, w_pde, d_pde)

def generate_bc_points_parametric(N_bc, L_n, w_n, d_n, device, L_char_x, L_char_y):
    scale_ratio = L_char_x / L_char_y
    w_n_y_scaled = w_n * scale_ratio
    d_n_y_scaled = d_n * scale_ratio
    n_plates_total = N_bc // 2; n_neumann_total = N_bc - n_plates_total
    n_per_plate = n_plates_total // 2; n_plate_side = max(1, n_per_plate // 4)
    n_outer_side = max(1, n_neumann_total // 4)
    plate_x_range_n_dyn = [-L_n / 2, L_n / 2]
    top_plate_y_range_n_dyn = [d_n_y_scaled / 2, d_n_y_scaled / 2 + w_n_y_scaled]
    bottom_plate_y_range_n_dyn = [-d_n_y_scaled / 2 - w_n_y_scaled, -d_n_y_scaled / 2]
    x_t_t = torch.rand(n_plate_side,1,device=device)*L_n+plate_x_range_n_dyn[0]; y_t_t=torch.ones_like(x_t_t)*top_plate_y_range_n_dyn[1]
    x_t_b = torch.rand(n_plate_side,1,device=device)*L_n+plate_x_range_n_dyn[0]; y_t_b=torch.ones_like(x_t_b)*top_plate_y_range_n_dyn[0]
    y_t_l=torch.rand(n_plate_side,1,device=device)*w_n_y_scaled+top_plate_y_range_n_dyn[0]; x_t_l=torch.ones_like(y_t_l)*plate_x_range_n_dyn[0]
    y_t_r=torch.rand(n_plate_side,1,device=device)*w_n_y_scaled+top_plate_y_range_n_dyn[0]; x_t_r=torch.ones_like(y_t_r)*plate_x_range_n_dyn[1]
    x_top=torch.cat([x_t_t,x_t_b,x_t_l,x_t_r]); y_top=torch.cat([y_t_t,y_t_b,y_t_l,y_t_r])
    x_b_t=torch.rand(n_plate_side,1,device=device)*L_n+plate_x_range_n_dyn[0]; y_b_t=torch.ones_like(x_b_t)*bottom_plate_y_range_n_dyn[1]
    x_b_b=torch.rand(n_plate_side,1,device=device)*L_n+plate_x_range_n_dyn[0]; y_b_b=torch.ones_like(x_b_b)*bottom_plate_y_range_n_dyn[0]
    y_b_l=torch.rand(n_plate_side,1,device=device)*w_n_y_scaled+bottom_plate_y_range_n_dyn[0]; x_b_l=torch.ones_like(y_b_l)*plate_x_range_n_dyn[0]
    y_b_r=torch.rand(n_plate_side,1,device=device)*w_n_y_scaled+bottom_plate_y_range_n_dyn[0]; x_b_r=torch.ones_like(y_b_r)*plate_x_range_n_dyn[1]
    x_bottom=torch.cat([x_b_t,x_b_b,x_b_l,x_b_r]); y_bottom=torch.cat([y_b_t,y_b_b,y_b_l,y_b_r])
    x_n_l=torch.full((n_outer_side,1),-1.0,device=device); y_n_l=torch.rand(n_outer_side,1,device=device)*2-1
    x_n_r=torch.full((n_outer_side,1),1.0,device=device); y_n_r=torch.rand(n_outer_side,1,device=device)*2-1
    y_n_b=torch.full((n_outer_side,1),-1.0,device=device); x_n_b=torch.rand(n_outer_side,1,device=device)*2-1
    y_n_t=torch.full((n_outer_side,1),1.0,device=device); x_n_t=torch.rand(n_outer_side,1,device=device)*2-1
    x_neumann=torch.cat([x_n_l,x_n_r,x_n_b,x_n_t]); y_neumann=torch.cat([y_n_l,y_n_r,y_n_b,y_n_t])
    return {'top_plate': (x_top, y_top), 'bottom_plate': (x_bottom, y_bottom), 'neumann': (x_neumann, y_neumann)}

In [ ]:
def compute_loss_parametric(model, pde_points_n, bc_points_n, corrective_data, params_dict, E_norm_factor):
    x_pde_n, y_pde_n = pde_points_n
    L_pde_n, w_pde_n, d_pde_n = params_dict['pde']
    x_pde_n.requires_grad_(True); y_pde_n.requires_grad_(True)
    V_pde = model(x_pde_n, y_pde_n, L_pde_n, w_pde_n, d_pde_n)
    first_grads_pde = torch.autograd.grad(V_pde.sum(), [x_pde_n, y_pde_n], create_graph=True)
    V_x_n, V_y_n = first_grads_pde[0], first_grads_pde[1]
    V_xx_n = torch.autograd.grad(V_x_n.sum(), x_pde_n, retain_graph=True)[0]
    V_yy_n = torch.autograd.grad(V_y_n.sum(), y_pde_n)[0]
    loss_pde = torch.mean((V_xx_n + V_yy_n)**2)
    L_bc_n, w_bc_n, d_bc_n = params_dict['bc']
    x_top_n, y_top_n = bc_points_n['top_plate']
    loss_bc_top = torch.mean((model(x_top_n, y_top_n, L_bc_n, w_bc_n, d_bc_n) - V_top)**2)
    x_bottom_n, y_bottom_n = bc_points_n['bottom_plate']
    loss_bc_bottom = torch.mean((model(x_bottom_n, y_bottom_n, L_bc_n, w_bc_n, d_bc_n) - V_bottom)**2)
    loss_bc_dirichlet = loss_bc_top + loss_bc_bottom
    x_n, y_n = bc_points_n['neumann']
    x_n.requires_grad_(True); y_n.requires_grad_(True)
    V_n = model(x_n, y_n, L_bc_n, w_bc_n, d_bc_n)
    first_grads_neu = torch.autograd.grad(V_n.sum(), [x_n, y_n], create_graph=False)
    V_nx_n, V_ny_n = first_grads_neu[0], first_grads_neu[1]
    is_lr = (torch.abs(x_n - x_domain_n[0]) < 1e-6) | (torch.abs(x_n - x_domain_n[1]) < 1e-6)
    is_tb = (torch.abs(y_n - y_domain_n[0]) < 1e-6) | (torch.abs(y_n - y_domain_n[1]) < 1e-6)
    loss_neumann = torch.mean(V_nx_n[is_lr]**2) + torch.mean(V_ny_n[is_tb]**2)
    x_corr_n, y_corr_n, v_corr, ex_corr_n_scaled, ey_corr_n_scaled, L_corr_n, w_corr_n, d_corr_n = corrective_data
    if x_corr_n.shape[0] > 0:
        x_corr_n.requires_grad_(True); y_corr_n.requires_grad_(True)
        v_pred_corr = model(x_corr_n, y_corr_n, L_corr_n, w_corr_n, d_corr_n)
        loss_data = torch.mean((v_pred_corr - v_corr)**2)
        grads_v_corr = torch.autograd.grad(v_pred_corr.sum(), [x_corr_n, y_corr_n], create_graph=True)
        dvdx_pred_n, dvdy_pred_n = grads_v_corr[0], grads_v_corr[1]
        ex_pred_n = -dvdx_pred_n; ey_pred_n = -dvdy_pred_n
        ex_pred_n_scaled = ex_pred_n / E_norm_factor
        ey_pred_n_scaled = ey_pred_n / E_norm_factor
        mse_ex = torch.mean((ex_pred_n_scaled - ex_corr_n_scaled)**2)
        mse_ey = torch.mean((ey_pred_n_scaled - ey_corr_n_scaled)**2)
        loss_E = W_E_FIELD_X * mse_ex + W_E_FIELD_Y * mse_ey
    else:
        loss_data = torch.tensor(0.0, device=device); loss_E = torch.tensor(0.0, device=device)
    v_pred_all_pde = model(x_pde_n, y_pde_n, L_pde_n, w_pde_n, d_pde_n)
    loss_overshoot = torch.mean(torch.relu(v_pred_all_pde - 1.0) + torch.relu(-v_pred_all_pde))
    loss_total = loss_pde + W_DIRICHLET * loss_bc_dirichlet + W_NEUMANN * loss_neumann + \
                 W_OVERSHOOT * loss_overshoot + W_DATA * loss_data + loss_E
    return loss_total, loss_pde, loss_bc_dirichlet, loss_neumann, loss_overshoot, loss_data, loss_E

In [ ]:
train_param_sets = [
    {'L': 40, 'w': 2, 'd': 2}, {'L': 40, 'w': 3.3, 'd': 2}, {'L': 40, 'w': 4.7, 'd': 2}, {'L': 40, 'w': 6, 'd': 2},
    {'L': 63.3, 'w': 2, 'd': 2}, {'L': 63.3, 'w': 3.3, 'd': 2}, {'L': 63.3, 'w': 4.7, 'd': 2}, {'L': 63.3, 'w': 6, 'd': 2},
    {'L': 86.7, 'w': 2, 'd': 2}, {'L': 86.7, 'w': 3.3, 'd': 2}, {'L': 86.7, 'w': 4.7, 'd': 2}, {'L': 86.7, 'w': 6, 'd': 2},
    {'L': 110, 'w': 2, 'd': 2}, {'L': 110, 'w': 3.3, 'd': 2}, {'L': 110, 'w': 4.7, 'd': 2}, {'L': 110, 'w': 6, 'd': 2},
    {'L': 40, 'w': 2, 'd': 4}, {'L': 40, 'w': 3.3, 'd': 4}, {'L': 40, 'w': 4.7, 'd': 4}, {'L': 40, 'w': 6, 'd': 4},
    {'L': 63.3, 'w': 2, 'd': 4}, {'L': 63.3, 'w': 3.3, 'd': 4}, {'L': 63.3, 'w': 4.7, 'd': 4}, {'L': 63.3, 'w': 6, 'd': 4},
    {'L': 86.7, 'w': 2, 'd': 4}, {'L': 86.7, 'w': 3.3, 'd': 4}, {'L': 86.7, 'w': 4.7, 'd': 4}, {'L': 86.7, 'w': 6, 'd': 4},
    {'L': 110, 'w': 2, 'd': 4}, {'L': 110, 'w': 3.3, 'd': 4}, {'L': 110, 'w': 4.7, 'd': 4}, {'L': 110, 'w': 6, 'd': 4},
    {'L': 40, 'w': 2, 'd': 6}, {'L': 40, 'w': 3.3, 'd': 6}, {'L': 40, 'w': 4.7, 'd': 6}, {'L': 40, 'w': 6, 'd': 6},
    {'L': 63.3, 'w': 2, 'd': 6}, {'L': 63.3, 'w': 3.3, 'd': 6}, {'L': 63.3, 'w': 4.7, 'd': 6}, {'L': 63.3, 'w': 6, 'd': 6},
    {'L': 86.7, 'w': 2, 'd': 6}, {'L': 86.7, 'w': 3.3, 'd': 6}, {'L': 86.7, 'w': 4.7, 'd': 6}, {'L': 86.7, 'w': 6, 'd': 6},
    {'L': 110, 'w': 2, 'd': 6}, {'L': 110, 'w': 3.3, 'd': 6}, {'L': 110, 'w': 4.7, 'd': 6}, {'L': 110, 'w': 6, 'd': 6},
    {'L': 40, 'w': 2, 'd': 8}, {'L': 40, 'w': 3.3, 'd': 8}, {'L': 40, 'w': 4.7, 'd': 8}, {'L': 40, 'w': 6, 'd': 8},
    {'L': 63.3, 'w': 2, 'd': 8}, {'L': 63.3, 'w': 3.3, 'd': 8}, {'L': 63.3, 'w': 4.7, 'd': 8}, {'L': 63.3, 'w': 6, 'd': 8},
    {'L': 86.7, 'w': 2, 'd': 8}, {'L': 86.7, 'w': 3.3, 'd': 8}, {'L': 86.7, 'w': 4.7, 'd': 8}, {'L': 86.7, 'w': 6, 'd': 8},
    {'L': 110, 'w': 2, 'd': 8}, {'L': 110, 'w': 3.3, 'd': 8}, {'L': 110, 'w': 4.7, 'd': 8}, {'L': 110, 'w': 6, 'd': 8},
]

all_dataframes = []
for params in train_param_sets:
    L_val, w_val, d_val = params['L'], params['w'], params['d']
    d_val_str = str(d_val).replace('.', '_')
    filename = f"comsol_V_Ex_Ey_C_L{L_val}_w{w_val}_d{d_val_str}.csv"
    try:
        df = pd.read_csv(filename); df['L'] = L_val; df['w'] = w_val; df['d'] = d_val
        all_dataframes.append(df)
        print(f"Loaded successfully: {filename}")
    except FileNotFoundError:
        print(f"Not found: {filename}")

if not all_dataframes:
    print("Failed to load any data files"); exit()
master_df = pd.concat(all_dataframes, ignore_index=True)
df_cleaned = master_df.dropna()

x_db_phys = torch.tensor(df_cleaned['X'].values,dtype=torch.float32).view(-1,1).to(device)
y_db_phys = torch.tensor(df_cleaned['Y'].values,dtype=torch.float32).view(-1,1).to(device)
v_db = torch.tensor(df_cleaned['Potential'].values,dtype=torch.float32).view(-1,1).to(device)
L_db_phys = torch.tensor(df_cleaned['L'].values,dtype=torch.float32).view(-1,1).to(device)
w_db_phys = torch.tensor(df_cleaned['w'].values,dtype=torch.float32).view(-1,1).to(device)
d_db_phys = torch.tensor(df_cleaned['d'].values,dtype=torch.float32).view(-1,1).to(device)
ex_db_phys = torch.tensor(df_cleaned['Ex'].values,dtype=torch.float32).view(-1,1).to(device)
ey_db_phys = torch.tensor(df_cleaned['Ey'].values,dtype=torch.float32).view(-1,1).to(device)

x_db_n = x_db_phys / L_char_x; y_db_n = y_db_phys / L_char_y
L_db_n = L_db_phys / L_char_x; w_db_n = w_db_phys / L_char_x; d_db_n = d_db_phys / L_char_x
ex_db_n = ex_db_phys * L_char_x; ey_db_n = ey_db_phys * L_char_y

E_norm_factor = max(torch.abs(ex_db_n).max(), torch.abs(ey_db_n).max())
ex_db_n_scaled = ex_db_n / E_norm_factor
ey_db_n_scaled = ey_db_n / E_norm_factor
print(f"E_norm_factor: {E_norm_factor.item()}")
print(f"Total number of database points: {len(df_cleaned)}")

In [ ]:
pinn_model = ParametricPINN(num_layers=NUM_LAYERS, hidden_size=HIDDEN_SIZE).to(device)
optimizer = torch.optim.Adam(pinn_model.parameters(), lr=LEARNING_RATE)

start_iteration = 0
corrective_set_n = torch.empty((0, 8), device=device)
checkpoint_path = 'training_checkpoint.pth'

if os.path.exists(checkpoint_path):
    print(f"Found breakpoint file: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path)
    pinn_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_iteration = checkpoint['iteration']
    corrective_set_n = checkpoint['corrective_set_n'].to(device)
    print(f"Continue training from {start_iteration + 1} iteration")

total_start_time = time.time()
for iteration in range(start_iteration, N_ITERATIONS):
    print(f"\n{'='*20} Number of iterations {iteration+1}/{N_ITERATIONS} {'='*20}")
    
    pinn_model.train()
    iter_start_time = time.time()
    for epoch in range(EPOCHS_PER_ITERATION):
        pde_points_n, pde_params_n = generate_pde_points_parametric(N_PDE_POINTS_PARAMETRIC, device, L_char_x, L_char_y)
        L_bc_n, w_bc_n, d_bc_n = sample_params(1, device, L_char_x)
        bc_points_n = generate_bc_points_parametric(N_BC_POINTS_PARAMETRIC, L_bc_n, w_bc_n, d_bc_n, device, L_char_x, L_char_y)
        params_dict = {'pde': pde_params_n, 'bc': (L_bc_n, w_bc_n, d_bc_n)}

        if corrective_set_n.shape[0] > 0:
            sample_size = min(N_ADD_POINTS, corrective_set_n.shape[0])
            sample_indices = torch.randint(0, corrective_set_n.shape[0], (sample_size,))
            corrective_batch = corrective_set_n[sample_indices]
            x_corr_n, y_corr_n, v_corr, ex_corr_n_scaled, ey_corr_n_scaled, L_corr_n, w_corr_n, d_corr_n = torch.split(corrective_batch, 1, dim=1)
        else:
            x_corr_n, y_corr_n, v_corr, ex_corr_n_scaled, ey_corr_n_scaled, L_corr_n, w_corr_n, d_corr_n = [torch.empty((0,1), device=device)] * 8
        corrective_data = (x_corr_n, y_corr_n, v_corr, ex_corr_n_scaled, ey_corr_n_scaled, L_corr_n, w_corr_n, d_corr_n)
        
        loss, pde, diri, neu, over, data, loss_e = compute_loss_parametric(pinn_model, pde_points_n, bc_points_n, corrective_data, params_dict, E_norm_factor)
        
        if torch.isnan(loss):
            print(f"Loss became NAN in the epoch {epoch},stop training")
            break
            
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(pinn_model.parameters(), max_norm=1.0)
        optimizer.step()

        if epoch % 1000 == 0:
            print(f"  Epoch {epoch:5d} | L:{loss.item():.2e} | PDE:{pde.item():.2e} | Diri:{diri.item():.2e} | Neu:{neu.item():.2e} | Over:{over.item():.2e} | Data:{data.item():.2e} | Loss_E:{loss_e.item():.2e}")
    
    if 'loss' in locals() and torch.isnan(loss):
        break
        
    print(f" This round of training took time: {time.time() - iter_start_time:.2f} seconds")

    pinn_model.eval()
    
    v_pred_db_list = []
    batch_size = 16384
    with torch.no_grad():
        for i in range(0, x_db_n.shape[0], batch_size):
            v_pred_batch = pinn_model(x_db_n[i:i+batch_size], y_db_n[i:i+batch_size],
                                      L_db_n[i:i+batch_size], w_db_n[i:i+batch_size], d_db_n[i:i+batch_size])
            v_pred_db_list.append(v_pred_batch)
    v_pred_db = torch.cat(v_pred_db_list, dim=0)
    
    errors = torch.abs(v_pred_db - v_db)
    max_error = torch.max(errors)
    print(f" Current Maximum Error: {max_error.item():.4f} V")
    
    _, indices = torch.topk(errors.flatten(), N_ADD_POINTS)
    new_points_to_add = torch.cat([
        x_db_n[indices], y_db_n[indices], v_db[indices],
        ex_db_n_scaled[indices], ey_db_n_scaled[indices],
        L_db_n[indices], w_db_n[indices], d_db_n[indices]
    ], dim=1)
    
    corrective_set_n = torch.cat([corrective_set_n, new_points_to_add])
    
    unique_points = torch.unique(corrective_set_n.cpu(), dim=0).to(device)
    corrective_set_n = unique_points
    
    print(f" New point has been added to the correction set.The current size of the correction set is: {corrective_set_n.shape[0]}")
    
    print("Save the current training progress to the checkpoint file")
    torch.save({
        'iteration': iteration + 1,
        'model_state_dict': pinn_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'corrective_set_n': corrective_set_n.cpu(),
    }, checkpoint_path)
    print("Checkpoint saved successfully")

print(f"\ntotal training time spent: {time.time() - total_start_time:.2f} seconds")
torch.save(pinn_model.state_dict(), 'final_parametric_model.pth')
print("The model has been saved to'final_parametric_model.pth'")

In [ ]:
def preload_comsol_data(param_sets_to_load, device):
    ground_truth_data = {}
    print("Preloading comsol data")
    for params in param_sets_to_load:
        L, w, d = params['L'], params['w'], params['d']
        #d_str = str(d).replace('.', '_')
        filename = f"comsol_V_Ex_Ey_C_L{L}_w{w}_d{d}.csv"
        try:
            df = pd.read_csv(filename)
            if len(df) != 80601:
                print(f"the file {filename} line count is not 80601")
                continue
            nx, ny = 401, 201
            v_matrix = df['Potential'].values.reshape((ny, nx), order='F')
            ex_matrix = df['Ex'].values.reshape((ny, nx), order='F')
            ey_matrix = df['Ey'].values.reshape((ny, nx), order='F')
            v_comsol = torch.tensor(v_matrix, dtype=torch.float32).to(device)
            ex_comsol = torch.tensor(ex_matrix, dtype=torch.float32).to(device)
            ey_comsol = torch.tensor(ey_matrix, dtype=torch.float32).to(device)
            c_comsol = df.dropna()['Capacitance_fF'].iloc[0]
            ground_truth_data[(L, w, d)] = {
                'V': v_comsol, 'Ex': ex_comsol, 'Ey': ey_comsol, 'C': c_comsol
            }
            print(f"Preload successfully: {filename}")
        except FileNotFoundError:
            print(f"Not found: {filename}")
        except Exception as e:
            print(f"Error while processing the file{filename} : {e}")
    return ground_truth_data

def customize_colorbar_ticks(cbar, data_array):
    clean_data = data_array[~np.isnan(data_array)]
    if clean_data.size == 0: return
    min_val, max_val = np.min(clean_data), np.max(clean_data)
    default_ticks = cbar.get_ticks()
    new_ticks = {min_val, max_val}
    for tick in default_ticks:
        if min_val < tick < max_val: new_ticks.add(tick)
    sorted_ticks = sorted(list(new_ticks))
    cbar.set_ticks(sorted_ticks)
    cbar.ax.set_yticklabels([f'{t:.2e}' if abs(t) > 1000 else f'{t:.3f}' for t in sorted_ticks])

def is_inside_gap_parametric(x_n, y_n, L_n, d_n, L_char_x, L_char_y):
    scale_ratio = L_char_x / L_char_y
    d_n_y_scaled = d_n * scale_ratio
    gap_x_min_n = -L_n / 2
    gap_x_max_n = L_n / 2
    gap_y_min_n = -d_n_y_scaled / 2
    gap_y_max_n = d_n_y_scaled / 2
    return (x_n >= gap_x_min_n) & (x_n <= gap_x_max_n) & \
           (y_n >= gap_y_min_n) & (y_n <= gap_y_max_n)

def generate_points_for_fringe(n_points, L, w, d, device, L_char_x, L_char_y):
    L_n_val = L / L_char_x
    w_n_val = w / L_char_x
    d_n_val = d / L_char_x
    x_fringe_n_list, y_fringe_n_list = [], []
    num_generated = 0
    while num_generated < n_points:
        needed = n_points - num_generated
        num_candidates = int(needed * 2.0) + 100
        x_cand_n = torch.rand(num_candidates, 1, device=device) * 2 - 1
        y_cand_n = torch.rand(num_candidates, 1, device=device) * 2 - 1
        L_cand_n = torch.full_like(x_cand_n, L_n_val)
        w_cand_n = torch.full_like(x_cand_n, w_n_val)
        d_cand_n = torch.full_like(x_cand_n, d_n_val)
        is_outside_plates = ~is_inside_plates_parametric(x_cand_n, y_cand_n, L_cand_n, w_cand_n, d_cand_n, L_char_x, L_char_y)
        is_outside_gap = ~is_inside_gap_parametric(x_cand_n, y_cand_n, L_cand_n, d_cand_n, L_char_x, L_char_y)
        valid_mask = is_outside_plates & is_outside_gap
        x_fringe_n_list.append(x_cand_n[valid_mask])
        y_fringe_n_list.append(y_cand_n[valid_mask])
        num_generated += valid_mask.sum().item()
    x_fringe_n = torch.cat(x_fringe_n_list)[:n_points]
    y_fringe_n = torch.cat(y_fringe_n_list)[:n_points]
    return x_fringe_n, y_fringe_n

def calculate_capacitance_parametric_final(model, L, w, d, L_char_x, L_char_y):
    print(" Calculating capacitance")
    model.eval()
    epsilon_0 = 8.854187817e-12; epsilon = epsilon_0 * 1.0
    um_to_m = 1e-6; delta_V = 1.0
    Z_depth = 60; Z_depth_m = Z_depth * um_to_m
    L_char_x_m = L_char_x * um_to_m
    L_char_y_m = L_char_y * um_to_m
    L_n_test = torch.tensor([[L / L_char_x]], device=device)
    w_n_test = torch.tensor([[w / L_char_x]], device=device)
    d_n_test = torch.tensor([[d / L_char_x]], device=device)
    n_energy_samples = 2000000
    n_gap_samples = int(n_energy_samples * 0.8)
    n_fringe_samples = n_energy_samples - n_gap_samples
    gap_area_m2 = (L * um_to_m) * (d * um_to_m)
    total_air_area_m2 = (lx * um_to_m * ly * um_to_m) - (L * um_to_m * w * um_to_m * 2)
    fringe_area_m2 = total_air_area_m2 - gap_area_m2
    if fringe_area_m2 <= 0: fringe_area_m2 = 1e-12
    L_n_val, d_n_val = L / L_char_x, d / L_char_x
    gap_x_min_n, gap_x_max_n = -L_n_val / 2, L_n_val / 2
    gap_y_min_n_mapped = (-d / 2) / L_char_y
    gap_y_max_n_mapped = (d / 2) / L_char_y
    x_gap_n = (torch.rand(n_gap_samples, 1, device=device) * (gap_x_max_n - gap_x_min_n) + gap_x_min_n).requires_grad_(True)
    y_gap_n = (torch.rand(n_gap_samples, 1, device=device) * (gap_y_max_n_mapped - gap_y_min_n_mapped) + gap_y_min_n_mapped).requires_grad_(True)
    x_fringe_n, y_fringe_n = generate_points_for_fringe(n_fringe_samples, L, w, d, device, L_char_x, L_char_y)
    x_fringe_n = x_fringe_n.view(-1, 1).requires_grad_(True)
    y_fringe_n = y_fringe_n.view(-1, 1).requires_grad_(True)

    def get_energy_sum(x_points, y_points):
        energy_sum = 0.0
        batch_size = 16384
        for i in range(0, x_points.shape[0], batch_size):
            x_batch_n, y_batch_n = x_points[i:i+batch_size], y_points[i:i+batch_size]
            V_batch = model(x_batch_n, y_batch_n, L_n_test, w_n_test, d_n_test)
            grads_n = torch.autograd.grad(V_batch.sum(), [x_batch_n, y_batch_n], create_graph=False)
            dVdx_n, dVdy_n = grads_n[0], grads_n[1]
            grad_V_sq_phys = (dVdx_n**2 / L_char_x_m**2) + (dVdy_n**2 / L_char_y_m**2)
            u_batch = 0.5 * epsilon * grad_V_sq_phys
            energy_sum += torch.sum(u_batch).item()
        return energy_sum
    
    U_per_depth_gap = (get_energy_sum(x_gap_n, y_gap_n) / n_gap_samples if n_gap_samples > 0 else 0) * gap_area_m2
    U_per_depth_fringe = (get_energy_sum(x_fringe_n, y_fringe_n) / n_fringe_samples if n_fringe_samples > 0 else 0) * fringe_area_m2
    
    U_per_depth = U_per_depth_gap + U_per_depth_fringe
    Capacitance_from_U = (2 * U_per_depth * Z_depth_m) / (delta_V**2)
    return 0.0, Capacitance_from_U * 1e15

def analyze_gradient_model_case(model, L, w, d, ground_truth, L_char_x, L_char_y):
    print(f"\n--- analyse L={L}, w={w}, d={d} ")
    model.eval()

    gt = ground_truth.get((L, w, d))
    if gt is None:
        print("No comsol data in preload")
        return
    V_comsol, Ex_comsol, Ey_comsol, C_comsol_fF = gt['V'], gt['Ex'], gt['Ey'], gt['C']

    nx, ny = 401, 201
    x_plot_phys = torch.linspace(x_domain_phys[0], x_domain_phys[1], nx, device=device)
    y_plot_phys = torch.linspace(y_domain_phys[0], y_domain_phys[1], ny, device=device)
    X_phys, Y_phys = torch.meshgrid(x_plot_phys, y_plot_phys, indexing='xy')
    
    X_n_plot = X_phys / L_char_x
    Y_n_plot = Y_phys / L_char_y
    X_n_plot.requires_grad_(True); Y_n_plot.requires_grad_(True)
    
    L_n_test = torch.tensor([[L / L_char_x]], device=device)
    w_n_test = torch.tensor([[w / L_char_x]], device=device)
    d_n_test = torch.tensor([[d / L_char_x]], device=device)

    V_pred_flat = model(X_n_plot.flatten().view(-1,1), Y_n_plot.flatten().view(-1,1), L_n_test, w_n_test, d_n_test)
    V_pred = V_pred_flat.reshape(ny, nx)
    
    grad_V_n = torch.autograd.grad(V_pred.sum(), [X_n_plot, Y_n_plot], create_graph=False)
    dVdx_n, dVdy_n = grad_V_n[0], grad_V_n[1]
    
    um_to_m = 1e-6
    Ex_pred_phys = -dVdx_n / (L_char_x * um_to_m)
    Ey_pred_phys = -dVdy_n / (L_char_y * um_to_m)

    X_np, Y_np = X_phys.cpu().detach().numpy(), Y_phys.cpu().detach().numpy()
    V_pred_np, Ex_pred_np, Ey_pred_np = V_pred.cpu().detach().numpy(), Ex_pred_phys.cpu().detach().numpy(), Ey_pred_phys.cpu().detach().numpy()
    V_comsol_np, Ex_comsol_np, Ey_comsol_np = V_comsol.cpu().numpy(), Ex_comsol.cpu().numpy(), Ey_comsol.cpu().numpy()

    L_n = torch.tensor(L / L_char_x); w_n = torch.tensor(w / L_char_x); d_n = torch.tensor(d / L_char_x)
    is_in_air_mask = ~is_inside_plates_parametric(torch.from_numpy(X_n_plot.cpu().detach().numpy()), torch.from_numpy(Y_n_plot.cpu().detach().numpy()), L_n, w_n, d_n, L_char_x, L_char_y).numpy()
    
    
    error_V = np.abs(V_pred_np - V_comsol_np)
    error_Ex = np.abs(Ex_pred_np - Ex_comsol_np)
    error_Ey = np.abs(Ey_pred_np - Ey_comsol_np)
    
    plate_x_range = [-L/2, L/2]
    top_plate_y_range = [d/2, d/2+w]
    bottom_plate_y_range = [-d/2-w, -d/2]

    plot_tasks = [
        {'name': ' V', 'pred': V_pred_np, 'gt': V_comsol_np, 'err': error_V, 'label': 'Potential (V)', 'err_cmap': 'viridis'},
        {'name': ' Ex', 'pred': Ex_pred_np, 'gt': Ex_comsol_np, 'err': error_Ex, 'label': 'Ex (V/m)', 'err_cmap': 'magma'},
        {'name': ' Ey', 'pred': Ey_pred_np, 'gt': Ey_comsol_np, 'err': error_Ey, 'label': 'Ey (V/m)', 'err_cmap': 'plasma'}
    ]

    for task in plot_tasks:
        fig, axes = plt.subplots(1, 3, figsize=(21, 6), constrained_layout=True)
        fig.suptitle(f'{task["name"]} Precision Analysis (L={L}, w={w}, d={d})', fontsize=16)
        pred_data_air_only = np.where(is_in_air_mask, task['pred'], np.nan)
        gt_data_air_only = task['gt']
        error_data = np.abs(pred_data_air_only - gt_data_air_only)
        vmin = np.nanmin(gt_data_air_only)
        vmax = np.nanmax(gt_data_air_only)
        ax = axes[0]; ax.set_title(f'PINN Prediction'); ax.set_ylabel('y (μm)'); ax.set_xlabel('x (μm)')
        im = ax.pcolor(X_np, Y_np, pred_data_air_only, cmap='jet', shading='auto', vmin=vmin, vmax=vmax)
        cbar = fig.colorbar(im, ax=ax, shrink=0.8); cbar.set_label(task['label']); customize_colorbar_ticks(cbar, pred_data_air_only)
        if 'Ex' in task['name']: clim = np.nanmax(np.abs(gt_data_air_only)); cbar.mappable.set_clim(-clim, clim)
        ax = axes[1]; ax.set_title(f'COMSOL Ground Truth'); ax.set_xlabel('x (μm)')
        im = ax.pcolor(X_np, Y_np, gt_data_air_only, cmap='jet', shading='auto', vmin=vmin, vmax=vmax)
        cbar = fig.colorbar(im, ax=ax, shrink=0.8); cbar.set_label(task['label']); customize_colorbar_ticks(cbar, gt_data_air_only)
        if 'Ex' in task['name']: clim = np.nanmax(np.abs(gt_data_air_only)); cbar.mappable.set_clim(-clim, clim)
        ax = axes[2]; ax.set_title(f'Absolute Error'); ax.set_xlabel('x (μm)')
        im = ax.pcolor(X_np, Y_np, error_data, cmap=task['err_cmap'], shading='auto')
        cbar = fig.colorbar(im, ax=ax, shrink=0.8); cbar.set_label(f'Error'); customize_colorbar_ticks(cbar, error_data)

        for ax_ in axes:
             ax_.add_patch(plt.Rectangle((-L/2, d/2), L, w, fill=True, color='gray', zorder=10))
             ax_.add_patch(plt.Rectangle((-L/2, -d/2-w), L, w, fill=True, color='gray', zorder=10))
        plt.show()

    _, C_pinn_U_fF = calculate_capacitance_parametric_final(model, L, w, d, L_char_x, L_char_y)
    error_U = 100 * abs(C_pinn_U_fF - C_comsol_fF) / C_comsol_fF
    
    print("\n" + "-"*20 + f" Capacitance Analysis (L={L}, w={w}, d={d}) " + "-"*20)
    print(f"  COMSOL Ground Truth: {C_comsol_fF:.4f} fF")
    print(f"  PINN Predicted Capacitance: {C_pinn_U_fF:.4f} fF (Error: {error_U:.2f} %)")
    print("-" * (60 + len(f" (L={L}, w={w}, d={d}) ")))

In [ ]:
train_param_sets = [
    {'L': 40, 'w': 2, 'd': 2}, {'L': 40, 'w': 3.3, 'd': 2}, {'L': 40, 'w': 4.7, 'd': 2}, {'L': 40, 'w': 6, 'd': 2},
    {'L': 63.3, 'w': 2, 'd': 2}, {'L': 63.3, 'w': 3.3, 'd': 2}, {'L': 63.3, 'w': 4.7, 'd': 2}, {'L': 63.3, 'w': 6, 'd': 2},
    {'L': 86.7, 'w': 2, 'd': 2}, {'L': 86.7, 'w': 3.3, 'd': 2}, {'L': 86.7, 'w': 4.7, 'd': 2}, {'L': 86.7, 'w': 6, 'd': 2},
    {'L': 110, 'w': 2, 'd': 2}, {'L': 110, 'w': 3.3, 'd': 2}, {'L': 110, 'w': 4.7, 'd': 2}, {'L': 110, 'w': 6, 'd': 2},
    {'L': 40, 'w': 2, 'd': 4}, {'L': 40, 'w': 3.3, 'd': 4}, {'L': 40, 'w': 4.7, 'd': 4}, {'L': 40, 'w': 6, 'd': 4},
    {'L': 63.3, 'w': 2, 'd': 4}, {'L': 63.3, 'w': 3.3, 'd': 4}, {'L': 63.3, 'w': 4.7, 'd': 4}, {'L': 63.3, 'w': 6, 'd': 4},
    {'L': 86.7, 'w': 2, 'd': 4}, {'L': 86.7, 'w': 3.3, 'd': 4}, {'L': 86.7, 'w': 4.7, 'd': 4}, {'L': 86.7, 'w': 6, 'd': 4},
    {'L': 110, 'w': 2, 'd': 4}, {'L': 110, 'w': 3.3, 'd': 4}, {'L': 110, 'w': 4.7, 'd': 4}, {'L': 110, 'w': 6, 'd': 4},
    {'L': 40, 'w': 2, 'd': 6}, {'L': 40, 'w': 3.3, 'd': 6}, {'L': 40, 'w': 4.7, 'd': 6}, {'L': 40, 'w': 6, 'd': 6},
    {'L': 63.3, 'w': 2, 'd': 6}, {'L': 63.3, 'w': 3.3, 'd': 6}, {'L': 63.3, 'w': 4.7, 'd': 6}, {'L': 63.3, 'w': 6, 'd': 6},
    {'L': 86.7, 'w': 2, 'd': 6}, {'L': 86.7, 'w': 3.3, 'd': 6}, {'L': 86.7, 'w': 4.7, 'd': 6}, {'L': 86.7, 'w': 6, 'd': 6},
    {'L': 110, 'w': 2, 'd': 6}, {'L': 110, 'w': 3.3, 'd': 6}, {'L': 110, 'w': 4.7, 'd': 6}, {'L': 110, 'w': 6, 'd': 6},
    {'L': 40, 'w': 2, 'd': 8}, {'L': 40, 'w': 3.3, 'd': 8}, {'L': 40, 'w': 4.7, 'd': 8}, {'L': 40, 'w': 6, 'd': 8},
    {'L': 63.3, 'w': 2, 'd': 8}, {'L': 63.3, 'w': 3.3, 'd': 8}, {'L': 63.3, 'w': 4.7, 'd': 8}, {'L': 63.3, 'w': 6, 'd': 8},
    {'L': 86.7, 'w': 2, 'd': 8}, {'L': 86.7, 'w': 3.3, 'd': 8}, {'L': 86.7, 'w': 4.7, 'd': 8}, {'L': 86.7, 'w': 6, 'd': 8},
    {'L': 110, 'w': 2, 'd': 8}, {'L': 110, 'w': 3.3, 'd': 8}, {'L': 110, 'w': 4.7, 'd': 8}, {'L': 110, 'w': 6, 'd': 8},
]

val_param_sets = [
    {'L': 75, 'w': 6, 'd': 8}, {'L': 90, 'w': 3, 'd': 4},
    {'L': 40, 'w': 2, 'd': 18}, {'L': 80, 'w': 10, 'd': 2},
    {'L': 80, 'w': 5, 'd': 5},
]
test_param_sets = [
    {'L': 100, 'w': 4, 'd': 2},  {'L': 90, 'w': 3, 'd': 2},
    {'L': 110, 'w': 4, 'd': 3},  {'L': 80, 'w': 3, 'd': 2.5},
]
sets_to_process = {
    "Training Set": train_param_sets,
    "Validation Set": val_param_sets,
    "Test Set": test_param_sets,
}

model_path = 'final_parametric_model.pth'
try:
    pinn_model_eval = ParametricPINN(num_layers=NUM_LAYERS, hidden_size=HIDDEN_SIZE).to(device)
    pinn_model_eval.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
    pinn_model_eval.eval()
    print("Model loaded successfully")

    for set_name, param_list in sets_to_process.items():
        print("\n" + "="*80 + f"\n     start verification.: {set_name}\n" + "="*80)
        all_truth_data = preload_comsol_data(param_list, device)
        if all_truth_data:
            for params in param_list:
                if (params['L'], params['w'], params['d']) in all_truth_data:
                    analyze_gradient_model_case(pinn_model_eval, L=params['L'], w=params['w'], d=params['d'], ground_truth=all_truth_data, L_char_x=L_char_x, L_char_y=L_char_y)
        else:
            print(f"No ground truth data for '{set_name}' ")
except NameError as e:
    print(f" NameError: {e}")
except FileNotFoundError:
    print(f" '{model_path}'not found")
except Exception as e:
    print(f"error: {e}")